## IMPORT LIBRARIES

In [1]:
import pandas as pd
from datetime import datetime

## DATA PREPARATION

### Read CSV

In [2]:
dfs=pd.read_csv("dataset/user_seller.csv")
dfb=pd.read_csv("dataset/user_buyer.csv")
dfo=pd.read_csv("dataset/order.csv")
dfod=pd.read_csv("dataset/order_details.csv")

C:\Users\stamp\AppData\Local\Temp\ipykernel_12292\3787394412.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dfo=pd.read_csv("dataset/order.csv")


### Initial Order Table

In [3]:
dfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159133 entries, 0 to 159132
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   po_number            159133 non-null  object 
 1   transaction_date     159133 non-null  object 
 2   order_address_id     159133 non-null  int64  
 3   order_status         159133 non-null  object 
 4   payment_group        159133 non-null  object 
 5   payment_method       159133 non-null  object 
 6   shipping_agency      159133 non-null  object 
 7   shipping_cost        159133 non-null  int64  
 8   total_project_value  159133 non-null  int64  
 9   voucher_val          68399 non-null   float64
 10  voucher_code         5160 non-null    object 
 11  revenue              159133 non-null  int64  
 12  seller_id            159127 non-null  object 
 13  seller_category      159127 non-null  object 
 14  buyer_id             159133 non-null  object 
dtypes: float64(1), in

In [4]:
# Check unique values
field_1 = ['po_number', 'order_address_id', 'seller_id', 'buyer_id']
dfo[field_1].nunique()

po_number           159114
order_address_id    159112
seller_id             6661
buyer_id             12810
dtype: int64

### dfo1. Add is_voucher_used

In [6]:
dfo1 = dfo.copy()
dfo1['is_voucher_used'] = dfo1['voucher_code'].notnull()
dfo1[dfo1['is_voucher_used'] == False][['voucher_code', 'is_voucher_used']]

,voucher_code,is_voucher_used
0,NaN,False
1,NaN,False
2,NaN,False
3,NaN,False
4,NaN,False
...,...,...
159128,NaN,False
159129,NaN,False
159130,NaN,False
159131,NaN,False


In [7]:
dfo1[dfo1['is_voucher_used'] == True][['voucher_code', 'is_voucher_used']]

,voucher_code,is_voucher_used
106932,ICIPJUNI,True
106950,ICIPJUNI,True
106951,ICIPJUNI,True
106952,RICIPJUNI-yXr51p73,True
106953,RICIPJUNI-GeWdvv64,True
...,...,...
159041,SALEBRASI25,True
159045,SALEBRASI25,True
159074,BRIBP227,True
159113,BRIBP227,True


In [8]:
# Count the number of null values in 'voucher_code'
null_voucher_count = dfo1['voucher_code'].isnull().sum()

# Count the number of False values in 'is_voucher_used'
false_is_voucher_count = (dfo1['is_voucher_used'] == False).sum()

# Check if the counts are equal
if null_voucher_count == false_is_voucher_count:
    print("The number of null values in 'voucher_code' is equal to the number of False values in 'is_voucher_used'.")
else:
    print("The counts do not match.")

The number of null values in 'voucher_code' is equal to the number of False values in 'is_voucher_used'.


In [9]:
dfo1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159133 entries, 0 to 159132
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   po_number            159133 non-null  object 
 1   transaction_date     159133 non-null  object 
 2   order_address_id     159133 non-null  int64  
 3   order_status         159133 non-null  object 
 4   payment_group        159133 non-null  object 
 5   payment_method       159133 non-null  object 
 6   shipping_agency      159133 non-null  object 
 7   shipping_cost        159133 non-null  int64  
 8   total_project_value  159133 non-null  int64  
 9   voucher_val          68399 non-null   float64
 10  voucher_code         5160 non-null    object 
 11  revenue              159133 non-null  int64  
 12  seller_id            159127 non-null  object 
 13  seller_category      159127 non-null  object 
 14  buyer_id             159133 non-null  object 
 15  is_voucher_used  

### Left Inner Join with Seller

In [11]:
dfo2 = dfo1.copy()
dfo2 = pd.merge(dfo1, dfs, left_on='seller_id', right_on='uid', how='left')

dfo2.rename(columns={'province': 'seller_prov', 'city': 'seller_city'}, inplace=True)
dfo2 = dfo2.drop(columns=['flag'])

dfo2.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159133 entries, 0 to 159132
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   po_number            159133 non-null  object 
 1   transaction_date     159133 non-null  object 
 2   order_address_id     159133 non-null  int64  
 3   order_status         159133 non-null  object 
 4   payment_group        159133 non-null  object 
 5   payment_method       159133 non-null  object 
 6   shipping_agency      159133 non-null  object 
 7   shipping_cost        159133 non-null  int64  
 8   total_project_value  159133 non-null  int64  
 9   voucher_val          68399 non-null   float64
 10  voucher_code         5160 non-null    object 
 11  revenue              159133 non-null  int64  
 12  seller_id            159127 non-null  object 
 13  seller_category      159127 non-null  object 
 14  buyer_id             159133 non-null  object 
 15  is_voucher_used  

### dfob. Group by buyer_id (Variable Selection)

In [12]:
# Convert 'transaction_date' column to datetime
dfo2['transaction_date'] = pd.to_datetime(dfo2['transaction_date'])

today = pd.to_datetime('today')
today = today.replace(year=2024, month=1, day=1)

In [13]:
# Define aggregation functions for each column
aggregations = {
    'transaction_date': 'max',  # latest transaction date
    'order_address_id': 'count',  # order frequency
    'total_project_value': 'sum',  # GMV
    'revenue': 'sum',  # revenue
    'is_voucher_used': 'sum',  # number of voucher used
    'voucher_val': 'sum',  # sum of voucher value
    'seller_id': 'nunique',  # count unique buyer_id
    'seller_prov': 'nunique',  # count unique buyer_province
    'seller_city': 'nunique',  # count unique buyer_city
}

# Group by 'seller_id' and apply the aggregation functions
dfob = dfo2.groupby('buyer_id').agg(aggregations)

# Calculate recency_day
dfob['recency_day'] = (today - dfob['transaction_date']).dt.days

# Calculate aov
dfob['aov'] = dfob['total_project_value'] / dfob['order_address_id']

# Rearrange columns as per your specified order
column_order = ['transaction_date', 'recency_day', 'order_address_id', 'total_project_value', 
                'aov', 'revenue', 'is_voucher_used', 'voucher_val', 'seller_id', 
                'seller_prov', 'seller_city']

dfob = dfob[column_order]

# Rename the columns
dfob.columns = ['last_transaction', 'recency_day', 'order_freq', 'gmv', 
                'aov', 'revenue', 'voucher_used', 'voucher_val', 'seller_count', 
                'num_of_seller_province', 'num_of_seller_city']

# Reset index to make 'seller_id' a regular column
dfob.reset_index(inplace=True)

dfob.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12810 entries, 0 to 12809
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   buyer_id                12810 non-null  object        
 1   last_transaction        12810 non-null  datetime64[ns]
 2   recency_day             12810 non-null  int64         
 3   order_freq              12810 non-null  int64         
 4   gmv                     12810 non-null  int64         
 5   aov                     12810 non-null  float64       
 6   revenue                 12810 non-null  int64         
 7   voucher_used            12810 non-null  int64         
 8   voucher_val             12810 non-null  float64       
 9   seller_count            12810 non-null  int64         
 10  num_of_seller_province  12810 non-null  int64         
 11  num_of_seller_city      12810 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(8), ob

In [14]:
dfob.head(25)

,buyer_id,last_transaction,recency_day,order_freq,gmv,aov,revenue,voucher_used,voucher_val,seller_count,num_of_seller_province,num_of_seller_city
0,631a4da47255a77e0e6d7410,2023-12-21,11,60,210014493,3.500242e+06,1575095,0,0.0,12,7,9
1,631a4f977255a77e0e6dec73,2023-12-29,3,110,2226297770,2.023907e+07,16697212,0,0.0,18,5,12
2,631a51497255a77e0e6eb585,2023-12-29,3,35,5215607524,1.490174e+08,19029058,0,0.0,15,6,12
3,631a514d7255a77e0e6eb6e5,2023-11-17,45,2,2370537,1.185268e+06,17779,0,0.0,1,1,1
4,631a514e7255a77e0e6eb785,2023-09-26,97,1,781000,7.810000e+05,5857,0,0.0,1,1,1
5,631a514f7255a77e0e6eb7c9,2023-12-31,1,769,44777116849,5.822772e+07,334952661,0,0.0,19,3,9
6,631a51507255a77e0e6eb83a,2023-09-25,98,2,18205664,9.102832e+06,136542,0,0.0,2,1,2
7,631a51527255a77e0e6eb8f5,2023-12-28,4,59,10932401,1.852949e+05,81980,0,0.0,28,9,23
8,631a51527255a77e0e6eb900,2023-12-20,12,24,124124367,5.171849e+06,930925,0,0.0,4,1,3
9,631a51557255a77e0e6eba0e,2023-01-25,341,1,7953000,7.953000e+06,59647,0,0.0,1,1,1


In [16]:
dfob['voucher_used'].value_counts()

voucher_used
0     9097
1     2810
2      781
3       55
4       15
6        9
5        8
7        4
8        4
9        4
12       3
23       3
10       3
15       3
22       2
11       2
18       2
24       1
14       1
21       1
19       1
13       1
Name: count, dtype: int64

### dfob_c. Saved to pickles

In [17]:
# Create a copy of dfos with 'seller_id' & 'last_transaction' columns removed
dfob_c = dfob.drop(columns=['buyer_id', 'last_transaction'])

# Save dfos_c as a pickle file
dfob_c.to_pickle('pickles/dfob_c.pkl')

# Optionally, you can also save dfos with the 'seller_id' column intact
dfob.to_pickle('pickles/dfob.pkl')